In [1]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [2]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [3]:
# Download the punkt tokenizer for sentence splitting
import nltk.data  

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences


In [4]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)
    
    "+= and appen() are different"

Parsing sentences from training set


/Users/yangjun/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/yangjun/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/Users/yangjun/anaconda/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/yangjun/anaconda/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks 

Parsing sentences from unlabeled set


/Users/yangjun/anaconda/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/yangjun/anaconda/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/yangjun/anaconda/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client 

In [5]:
print(len(sentences))

795538


In [6]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [10]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2017-11-06 19:51:03,166 : INFO : 'pattern' package not found; tag filters are not available for English
2017-11-06 19:51:03,171 : INFO : collecting all words and their counts
2017-11-06 19:51:03,172 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-11-06 19:51:03,252 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2017-11-06 19:51:03,300 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2017-11-06 19:51:03,344 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types


Training model...


2017-11-06 19:51:03,397 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2017-11-06 19:51:03,452 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2017-11-06 19:51:03,500 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2017-11-06 19:51:03,550 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2017-11-06 19:51:03,596 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2017-11-06 19:51:03,643 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2017-11-06 19:51:03,689 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2017-11-06 19:51:03,736 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2017-11-06 19:51:03,785 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

2017-11-06 19:51:07,458 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2017-11-06 19:51:07,530 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2017-11-06 19:51:07,602 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2017-11-06 19:51:07,673 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2017-11-06 19:51:07,714 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2017-11-06 19:51:07,715 : INFO : Loading a fresh vocabulary
2017-11-06 19:51:07,805 : INFO : min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2017-11-06 19:51:07,806 : INFO : min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2017-11-06 19:51:07,854 : INFO : deleting the raw counts dictionary of 123504 items
2017-11-06 19:51:07,860 : INFO : sample=0.001 d

2017-11-06 19:52:15,541 : INFO : PROGRESS: at 98.30% examples, 930416 words/s, in_qsize 7, out_qsize 0
2017-11-06 19:52:16,553 : INFO : PROGRESS: at 99.78% examples, 930439 words/s, in_qsize 7, out_qsize 0
2017-11-06 19:52:16,685 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-11-06 19:52:16,687 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-11-06 19:52:16,695 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-11-06 19:52:16,705 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-11-06 19:52:16,705 : INFO : training on 88990410 raw words (63752112 effective words) took 68.5s, 930455 effective words/s
2017-11-06 19:52:16,707 : INFO : precomputing L2-norms of word weight vectors
2017-11-06 19:52:16,846 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2017-11-06 19:52:16,847 : INFO : not storing attribute syn0norm
2017-11-06 19:52:16,848 : INFO : not storing att

In [11]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [12]:
model.doesnt_match("man")

'm'

In [13]:
model.most_similar("man")

[('woman', 0.6342664957046509),
 ('lady', 0.5826859474182129),
 ('lad', 0.5671110153198242),
 ('guy', 0.5402830243110657),
 ('farmer', 0.5364877581596375),
 ('chap', 0.5269407033920288),
 ('monk', 0.5266861915588379),
 ('millionaire', 0.5164661407470703),
 ('men', 0.5070557594299316),
 ('soldier', 0.5011072158813477)]